In [2]:
import re
from textblob import TextBlob
import string
#from senti_classifier import senti_classifier
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [199]:
hp1 = sc.textFile("file:/home/training/Desktop/HP5.txt")

In [200]:
book1RDD = hp1.map(lambda line: re.split('CHAPTER\s',line))

In [201]:
book1RDD.count()

1

In [202]:
test = book1RDD.take(1)

In [203]:
word_corpus = list()
for i in range(0,38):
    word_corpus.append(str(test[0][i].encode('utf-8')))

In [204]:
book1RDD = sc.parallelize(word_corpus)

In [205]:
def sentiment(word):
    sid = SentimentIntensityAnalyzer()
    for sentence in word:
        ss= sid.polarity_scores(sentence)
        for k in sorted(ss):
            print('{1}, '.format(ss[k]))

In [206]:
oneRDD = book1RDD.collect()
i = 1
for item in oneRDD:
    print("Word count for chapter {} - {}".format(i, len(item.split())))
    i += 1

Word count for chapter 1 - 5972
Word count for chapter 2 - 6260
Word count for chapter 3 - 5293
Word count for chapter 4 - 5796
Word count for chapter 5 - 5476
Word count for chapter 6 - 6977
Word count for chapter 7 - 4702
Word count for chapter 8 - 4334
Word count for chapter 9 - 8449
Word count for chapter 10 - 6057
Word count for chapter 11 - 6082
Word count for chapter 12 - 8557
Word count for chapter 13 - 9095
Word count for chapter 14 - 8370
Word count for chapter 15 - 7072
Word count for chapter 16 - 6149
Word count for chapter 17 - 6810
Word count for chapter 18 - 7190
Word count for chapter 19 - 6955
Word count for chapter 20 - 6641
Word count for chapter 21 - 7728
Word count for chapter 22 - 8257
Word count for chapter 23 - 7667
Word count for chapter 24 - 8362
Word count for chapter 25 - 8354
Word count for chapter 26 - 8928
Word count for chapter 27 - 7674
Word count for chapter 28 - 8302
Word count for chapter 29 - 7655
Word count for chapter 30 - 8557
Word count for chap

In [40]:
for chapter in range(1,book1RDD.count()+1):
    testimonial = TextBlob(str(book1RDD.take(chapter)))
    print("Chapter {}: The polarity is:{}".format(chapter,testimonial.sentiment.polarity))

Chapter 1: The polarity is:0.107235897717
Chapter 2: The polarity is:0.0441296518477
Chapter 3: The polarity is:0.0506001578271
Chapter 4: The polarity is:0.0640941837328
Chapter 5: The polarity is:0.0548952796252
Chapter 6: The polarity is:0.0604739047707
Chapter 7: The polarity is:0.0613165841573
Chapter 8: The polarity is:0.0637373626622
Chapter 9: The polarity is:0.0618598118639
Chapter 10: The polarity is:0.0542938317761
Chapter 11: The polarity is:0.0511414470719
Chapter 12: The polarity is:0.0503030758519
Chapter 13: The polarity is:0.0487875054626
Chapter 14: The polarity is:0.0466005709382
Chapter 15: The polarity is:0.0444610544916
Chapter 16: The polarity is:0.0434803673969
Chapter 17: The polarity is:0.043390712065


In [12]:
for chapter in range(1,book1RDD.count()+1):
    testimonial = TextBlob(str(book1RDD.take(chapter)))
    print("Chapter {}: The Subjectivity is:{}".format(chapter,testimonial.sentiment.subjectivity))

Chapter 1: The Subjectivity is:0.486919267469
Chapter 2: The Subjectivity is:0.477081046434
Chapter 3: The Subjectivity is:0.462337101336
Chapter 4: The Subjectivity is:0.48016922028
Chapter 5: The Subjectivity is:0.486131004085
Chapter 6: The Subjectivity is:0.471500111664
Chapter 7: The Subjectivity is:0.472537715987
Chapter 8: The Subjectivity is:0.474823787546
Chapter 9: The Subjectivity is:0.474243029188
Chapter 10: The Subjectivity is:0.475059651406
Chapter 11: The Subjectivity is:0.47583170769
Chapter 12: The Subjectivity is:0.474051054869
Chapter 13: The Subjectivity is:0.473377548682
Chapter 14: The Subjectivity is:0.472455413649
Chapter 15: The Subjectivity is:0.472846493106
Chapter 16: The Subjectivity is:0.474637351779
Chapter 17: The Subjectivity is:0.476261586719


In [207]:
abc = book1RDD.map(lambda x: x.split('.')) 

In [208]:
output = abc.collect()

In [209]:
a = []
for item in output:
    for x in item:
        words = x.split(" ")
        for word in words: 
            if len(word.strip()) > 0:
                a.append(word.rstrip().lstrip())

In [210]:
wordlistRDD = sc.parallelize(a)

In [211]:
cleanwordRDD = wordlistRDD.map(lambda z:re.sub(r'[^\s\w]','',z))

In [212]:
nltk.download('stopwords')
stop = set(stopwords.words('english'))
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/training/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/training/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [213]:
lem = WordNetLemmatizer()

In [214]:
removedstopwordsRDD = cleanwordRDD.filter(lambda word: word not in stop)

In [215]:
type(removedstopwordsRDD)

pyspark.rdd.PipelinedRDD

In [216]:
xyz = list()
for item in removedstopwordsRDD.collect():
    xyz.append(item.lower())

In [217]:
bgs = nltk.bigrams(xyz)

In [218]:
fdist = nltk.FreqDist(bgs)
#for k,v in fdist.items():
#    print (k,v)

In [219]:
sc.parallelize(fdist.items()).filter(lambda line: str(line[0][0]) not in ('said','','i') and str(line[0][0]) == 'privet').takeOrdered(20, key = lambda x: -x[1])

[(('privet', 'drive'), 24)]

In [220]:
sc.parallelize(fdist.items()).filter(lambda line: str(line[0][0]) not in ('said','','i') and str(line[0][0]) == 'diagon').takeOrdered(20, key = lambda x: -x[1])

[(('diagon', 'alley'), 7)]

In [221]:
sc.parallelize(fdist.items()).filter(lambda line: str(line[0][0]) not in ('said','','i') and str(line[0][0]) == 'grimmauld').takeOrdered(20, key = lambda x: -x[1])

[(('grimmauld', 'place'), 36)]

In [222]:
sc.parallelize(fdist.items()).filter(lambda line: str(line[0][0]) not in ('said','','i') and str(line[0][0]) == 'expecto').takeOrdered(20, key = lambda x: -x[1])

[(('expecto', 'patronum'), 4)]

In [262]:
sc.parallelize(fdist.items()).filter(lambda line: str(line[0][0]) not in ('said','','i')).takeOrdered(20, key = lambda x: -x[1])

[(('harry', 'ron'), 92),
 (('professor', 'mcgonagall'), 79),
 (('harry', 'potter'), 77),
 (('ron', 'hermione'), 55),
 (('harry', 'said'), 54),
 (('mrs', 'weasley'), 52),
 (('mr', 'weasley'), 48),
 (('fred', 'george'), 44),
 (('chamber', 'secrets'), 42),
 (('mr', 'malfoy'), 41),
 (('harry', 'could'), 38),
 (('uncle', 'vernon'), 34),
 (('harry', 'looked'), 33),
 (('nearly', 'headless'), 33),
 (('headless', 'nick'), 31),
 (('madam', 'pomfrey'), 28),
 (('gilderoy', 'lockhart'), 28),
 (('could', 'see'), 27),
 (('harry', ''), 27),
 (('ive', 'got'), 26)]

In [ ]:
from senti_classifier import senti_classifier
sentences = ['The movie was the worst movie', 'It was the worst acting by the actors']
pos_score, neg_score = senti_classifier.polarity_scores(sentences)
print pos_score, neg_score